In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('train.csv', header=0, sep='\t')
test = pd.read_csv('test.csv', header=0, sep=',')
data.shape

(237969, 386)

In [3]:
from sklearn import preprocessing, model_selection
from tqdm import tqdm

encoders = {}
for column in tqdm(test.columns[2:4]):
    if test.dtypes[column] == np.object:
        encoders[column] = preprocessing.LabelEncoder()
        fit_column = data[column].copy()
        fit_column = fit_column.append(test[column])
        encoders[column].fit(fit_column)
        test[column] = encoders[column].transform(test[column])
        data[column] = encoders[column].transform(data[column])
        
data = data.dropna()

100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


In [ ]:
print(data.head())

In [ ]:
print(test.head())

In [ ]:
print(test.shape, "    ", data.shape)

In [4]:
data = data.drop('DSSR', 1)
X_train = data.drop('mg', 1)
y_train = data['mg']
X_test = test.drop(test.columns[[0, 1, 3]], axis=1)

In [5]:
from sklearn.preprocessing import scale
#X_train = scale(np.array(X_train, dtype='float'), with_std=True, with_mean=True)
#X_test = scale(np.array(X_test, dtype='float'), with_std=True, with_mean=True)
X = np.vstack((X_train, X_test))
X = scale(np.array(X, dtype='float'), with_std=True, with_mean=True)
X_train = X[:X_train.shape[0]]
X_test = X[X_train.shape[0]:]

In [ ]:
print(X_test.shape, "    ", X_train.shape)

In [ ]:
print(X_train[:3])

In [ ]:
print(X_test[:3])

In [6]:
def write_answer(predictions):
    with open("result.csv", "w") as fout:
        fout.write("Id,mg\n")
        for i in range(predictions.shape[0]):
            fout.write(str(i)+","+str(predictions[i])+"\n")

In [ ]:
%%time
estimator = xgb.XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=3000, 
                              colsample_bylevel = X_train.shape[1]**(1/2)/X_train.shape[1],  n_jobs=-1)
estimator.fit(X_train, y_train)
predictions = estimator.predict(X_test)

In [ ]:
write_answer(predictions)

In [ ]:
print(np.argmax(estimator.feature_importances_))

In [ ]:
from xgboost import plot_importance
plot_importance(estimator)

In [7]:
from sklearn import cross_validation, grid_search, metrics

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [14]:
parameters_grid = {
    'learning_rate' : [0.05, 0.1, 0.2],
    'max_depth' : [2, 3, 4],
    'n_estimators' : [500],
    'colsample_bylevel' : [X_train.shape[1]**(1/2)/X_train.shape[1], 1],
}

In [10]:
cv = cross_validation.StratifiedShuffleSplit(y_train, n_iter = 10, test_size = 0.2)

In [11]:
classifier = xgb.XGBClassifier()

In [15]:
grid_cv = grid_search.GridSearchCV(classifier, parameters_grid, scoring = 'f1', cv = cv, n_jobs=4, pre_dispatch=8)

In [16]:
%%time
grid_cv.fit(X_train, y_train)

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in 

KeyboardInterrupt: 

In [17]:
grid_cv.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [18]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [ ]:
%%time
estimator = grid_cv.best_estimator_
estimator.fit(X_train, y_train)
predictions = estimator.predict(X_test)

In [ ]:
write_answer(predictions)

In [ ]:
print(np.argmax(estimator.feature_importances_))

In [ ]:
from xgboost import plot_importance
plot_importance(estimator)

In [ ]:
from sklearn import cross_validation

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
%%time
xgb_scoring = []
n_trees = [1] + list(range(100,200,50))
for n_tree in tqdm(n_trees):
    estimator = xgb.XGBClassifier(n_estimators=n_tree)
    score = cross_validation.cross_val_score(estimator, X_train, y_train, 
                                             scoring = 'F1', cv = 5)    
    xgb_scoring.append(score)
xgb_scoring = np.asmatrix(xgb_scoring)

In [ ]:
pylab.plot(n_trees, xgb_scoring.mean(axis = 1), marker='.', label='XGBoost')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')